In [1]:
from lxml import html

# from bs4 import BeautifulSoup
from bs4 import BeautifulSoup, NavigableString
import time
import datetime
import threading
from apscheduler.schedulers.background import BackgroundScheduler

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.chrome.options import Options

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec

from selenium.common.exceptions import TimeoutException

import helper

import json
import os
import sys

In [2]:
course_schedule = []

IMPLICIT_WAIT_TIME = 20
EXPLICIT_WAIT_TIME = 20


RETRY_INTERVAL = 65

In [3]:
from selenium.common.exceptions import NoSuchElementException        
def check_exists_by_xpath(xpath):
    try:
        driver.find_element("xpath",xpath)
    except NoSuchElementException:
        return False
    return True

def sync_get_element_by_xpath(driver, xpath):
    wait = WebDriverWait(driver, 15)
    try:
        wait.until(ec.visibility_of_element_located((By.XPATH, xpath)))
    except TimeoutException:
#         print("Get element from xpath:{} Timeout".format(xpath))
        return None
    return driver.find_element("xpath",xpath)

In [4]:
def teacher_norm(teacher):
    if not teacher.startswith('-'):
        return teacher
    tmp = teacher.split('\n')[0].split('- ')[1]
    return tmp[:-2]

def search_courses_by_date(date_str, courses):
    results = []
    
    date_str = date_str.strip()
    
    for item in courses:
        if date_str in item['date']:
            results.append(item)
    return results

def search_courses_by_teacher(teacher_str, courses):
    results = []
    
    teacher_str = teacher_str.strip()
    
    for item in courses:
        if teacher_str in item['teacher']:
            results.append(item)
    return results

def search_courses_by_name(name_str, courses):
    results = []
    
    name_str = name_str.strip()
    
    for item in courses:
        if name_str in item['name']:
            results.append(item)
    return results

def search_courses_by_time(time_str, courses):
    results = []
    
    time_str = time_str.strip()
    
    for item in courses:
        if time_str in item['time']:
            results.append(item)
    return results

    
def list_courses_to_be_booked(courses_to_be_booked):
    cur_date = ''
    if len(courses_to_be_booked) <= 0:
        return
    
    for course in courses_to_be_booked:  

        if cur_date != course['date']:
            if cur_date != '':
                print("-----------------------")
            print("Date: {}".format(course['date']))
            print("-----------------------\n")
            cur_date = course['date']
        print("Name: {}".format(course['name']))
        print("Teacher: {}".format(course['teacher']))
        print("Time: {}".format(course['time']))
        print("\n")
        

In [5]:
def create_new_driver():
    #start web driver

    from selenium import webdriver
    from selenium.webdriver.chrome.service import Service

    service = Service(executable_path=r'/usr/bin/chromedriver')

    #setup option for chrome profile
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')

    driver = webdriver.Chrome(service=service, options=options)

    driver.implicitly_wait(IMPLICIT_WAIT_TIME)
    wait = WebDriverWait(driver, EXPLICIT_WAIT_TIME)
    

    return driver, wait


In [6]:
def send_keys(elm, string):
    for c in string:
        elm.send_keys(c)
        
def sign_in(driver, wait):
    
    username = driver.find_element("id",'ctl00_cphContents_txtUsername')
    password = driver.find_element("id",'ctl00_cphContents_txtPassword')
    
    send_keys(username, "DN20092360")
    send_keys(password, "jj1216")
#     username.send_keys("DN23035242")
#     password.send_keys("NTW3BSFH")

    wait.until(ec.visibility_of_element_located((By.CLASS_NAME, "submit-button")))

    sign_in_btn = driver.find_element(By.CLASS_NAME,"submit-button")
    sign_in_btn.click()

def log_out(driver, wait):
    click_btn = helper.sync_get_element_by_xpath(driver, "//a[contains(text(), 'Logout')]")
#     print("click_btn: ", click_btn)
    if click_btn:
        click_btn.click()
    
def click_book_for_class():
    click_btn = sync_get_element_by_xpath(driver, "//a[contains(text(), '預定課')]")
#     print("click_btn: ", click_btn)
    click_btn.click()
    
def click_book_this_class_now(driver, wait):
    
    click_btn = sync_get_element_by_xpath(driver, "//a[contains(text(), '現在就預定此課')]")

    if click_btn:
        click_btn.click()
        return click_btn

In [7]:
#launch url
base_url = "https://www.trueclassbooking.com.tw/member/search-class.aspx"

In [8]:
temp = []

# load schedule from file
with open("./class_schedules/30 十月 - 05 十一月 2023.json", "r") as read_file:
    course_schedule = json.load(read_file)


Day_string = "03"

c = helper.search_courses_by_date(Day_string, course_schedule)
# c = helper.search_courses_by_name('Yin', c)
c = helper.search_courses_by_teacher('Una', c)
# c = helper.search_courses_by_time('PM', c)
temp.extend(c)

Day_string = "04"

c = helper.search_courses_by_date(Day_string, course_schedule)
# c = helper.search_courses_by_name('Mys', c)
c = helper.search_courses_by_teacher('Moor', c)
# c = helper.search_courses_by_time('PM', c)
temp.extend(c)


courses_to_be_booked = temp

list_courses_to_be_booked(courses_to_be_booked)

Date: 03 11月 周五
-----------------------

Name: Hatha Basic 哈達基礎
Teacher: Una
Time: 12:45 - 01:45 下午


Name: Yoga Wheel 瑜珈輪
Teacher: Una
Time: 03:30 - 04:30 下午


Name: Yoga Tone 2
Teacher: Una
Time: 02:00 - 03:00 下午


-----------------------
Date: 04 11月 周六
-----------------------

Name: Flow Yoga 動瑜珈
Teacher: Moorthi
Time: 11:00 - 12:00 下午


Name: Yin Yoga 陰瑜珈
Teacher: Moorthi
Time: 09:30 - 10:30 上午


Date: 03 11月 周五
-----------------------

Name: Hatha Basic 哈達基礎
Teacher: Una
Time: 12:45 - 01:45 下午


Name: Yoga Wheel 瑜珈輪
Teacher: Una
Time: 03:30 - 04:30 下午


Name: Yoga Tone 2
Teacher: Una
Time: 02:00 - 03:00 下午


-----------------------
Date: 04 11月 周六
-----------------------

Name: Flow Yoga 動瑜珈
Teacher: Moorthi
Time: 11:00 - 12:00 下午


Name: Yin Yoga 陰瑜珈
Teacher: Moorthi
Time: 09:30 - 10:30 上午




In [9]:
master_counter = 1

In [10]:
def sign_in_once(driver, wait):
#     print("sign_in({},{})".format(driver, wait))
    driver.get(base_url)
    sign_in(driver, wait)

In [11]:
def timestamp_string():

    date_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
#     date_time = datetime.datetime.now().strftime("%S")

    return date_time

In [12]:
csv_file_name = "./log_try_booking_a_class_that_was_full-- {}.csv".format(timestamp_string())

In [13]:
class_idx = 1
num_classes = len(courses_to_be_booked)

In [14]:
def reserve_class(driver, wait, course_to_be_booked, idx):
    
    global master_counter
    global class_idx
    global num_classes

    stride =RETRY_INTERVAL / 60 * 15 * num_classes

    book_url = course_to_be_booked['href']
    class_name = course_to_be_booked['name']
    teacher = course_to_be_booked['teacher']
    

    driver.get(book_url)

    click_btn=click_book_this_class_now(driver, wait)
    if click_btn is not None:
        
        print("master counter = {}, stride = {}, class_idx = {}".format(master_counter, stride, class_idx))

        log_str = "'{}','{}' by '{}',BOOKED,'{}'\n".format(master_counter, class_name, teacher, datetime.datetime.now())
        print(log_str)
        
        with open(csv_file_name, "a") as write_file:
            write_file.write(log_str)
        num_classes = num_classes - 1
    else:
        
        print("master counter = {}, stride = {}, class_idx = {}".format(master_counter, stride, class_idx))

        next_book_datetime = datetime.datetime.now() + datetime.timedelta(seconds=RETRY_INTERVAL)
        log_str = "'{}','{}' by '{}',FULL,'{}'\nnext_book_datetime={}\nclick_btn={}\n".format(master_counter, class_name, teacher, datetime.datetime.now(), next_book_datetime, click_btn)
        print(log_str)
        
        if 0==0: #(master_counter - class_idx) % stride == 0:
            
            with open(csv_file_name, "a") as write_file:
                write_file.write(log_str)
       
                
        driver.close()
        
        driver, wait = create_new_driver()

        sign_in_once(driver, wait)
        next_book_datetime = datetime.datetime.now() + datetime.timedelta(seconds=60+(idx%len(course_to_be_booked))*10 )
        print("now={}".format(datetime.datetime.now()))
        print("next_book_datetime={}".format(next_book_datetime))
        scheduler.add_job(reserve_class, 'date', run_date= next_book_datetime, args=[driver, wait, course_to_be_booked, idx])
    
        master_counter = master_counter + 1
    
    if class_idx < num_classes:
        class_idx = class_idx + 1
    else:
        class_idx = 1


In [15]:
drivers = []
waits = []
scheduler = BackgroundScheduler()
start = time.time()

#CSV file table header
with open(csv_file_name, "w") as write_file:
            write_file.write("#,Class,Status,Time\n")
        
for idx, course_to_be_booked in enumerate(courses_to_be_booked):

    driver, wait = create_new_driver()
    
    sign_in_once(driver, wait)
    next_book_datetime = datetime.datetime.now() + datetime.timedelta(seconds=60+(idx%len(course_to_be_booked))*10 )
    print("now={}".format(datetime.datetime.now()))
    print("next_book_datetime={}".format(next_book_datetime))
    scheduler.add_job(reserve_class, 'date', run_date= next_book_datetime, args=[driver, wait, course_to_be_booked, idx])
    
# print("Time Elapsed: {:.2f}".format(time.time() - start)) 
scheduler.start()

now=2023-11-01 14:27:40.254743
next_book_datetime=2023-11-01 14:28:40.254724
now=2023-11-01 14:27:40.254743
next_book_datetime=2023-11-01 14:28:40.254724
now=2023-11-01 14:27:45.340778
next_book_datetime=2023-11-01 14:28:55.340753
now=2023-11-01 14:27:45.340778
next_book_datetime=2023-11-01 14:28:55.340753
now=2023-11-01 14:27:50.014004
next_book_datetime=2023-11-01 14:29:10.013981
now=2023-11-01 14:27:50.014004
next_book_datetime=2023-11-01 14:29:10.013981
now=2023-11-01 14:27:54.642336
next_book_datetime=2023-11-01 14:29:24.642309
now=2023-11-01 14:27:54.642336
next_book_datetime=2023-11-01 14:29:24.642309
now=2023-11-01 14:27:59.446761
next_book_datetime=2023-11-01 14:29:39.446738
now=2023-11-01 14:27:59.446761
next_book_datetime=2023-11-01 14:29:39.446738
master counter = 1, stride = 81.25, class_idx = 1
'1','Hatha Basic 哈達基礎' by 'Una',FULL,'2023-11-01 14:29:00.950639'
next_book_datetime=2023-11-01 14:30:05.950625
click_btn=None

master counter = 1, stride = 81.25, class_idx = 1
'1

now=2023-11-01 14:33:42.001611
next_book_datetime=2023-11-01 14:35:12.001595
now=2023-11-01 14:33:42.001611
next_book_datetime=2023-11-01 14:35:12.001595
master counter = 16, stride = 81.25, class_idx = 1
'16','Yoga Wheel 瑜珈輪' by 'Una',FULL,'2023-11-01 14:34:04.170596'
next_book_datetime=2023-11-01 14:35:09.170579
click_btn=None

master counter = 16, stride = 81.25, class_idx = 1
'16','Yoga Wheel 瑜珈輪' by 'Una',FULL,'2023-11-01 14:34:04.170596'
next_book_datetime=2023-11-01 14:35:09.170579
click_btn=None

now=2023-11-01 14:34:09.200559
next_book_datetime=2023-11-01 14:35:19.200539
now=2023-11-01 14:34:09.200559
next_book_datetime=2023-11-01 14:35:19.200539
master counter = 17, stride = 81.25, class_idx = 2
'17','Yin Yoga 陰瑜珈' by 'Moorthi',FULL,'2023-11-01 14:34:11.656989'
next_book_datetime=2023-11-01 14:35:16.656967
click_btn=None

master counter = 17, stride = 81.25, class_idx = 2
'17','Yin Yoga 陰瑜珈' by 'Moorthi',FULL,'2023-11-01 14:34:11.656989'
next_book_datetime=2023-11-01 14:35:16

now=2023-11-01 14:39:05.252539
next_book_datetime=2023-11-01 14:40:05.252524
master counter = 32, stride = 81.25, class_idx = 2
'32','Flow Yoga 動瑜珈' by 'Moorthi',FULL,'2023-11-01 14:39:24.351207'
next_book_datetime=2023-11-01 14:40:29.351190
click_btn=None

master counter = 32, stride = 81.25, class_idx = 2
'32','Flow Yoga 動瑜珈' by 'Moorthi',FULL,'2023-11-01 14:39:24.351207'
next_book_datetime=2023-11-01 14:40:29.351190
click_btn=None

now=2023-11-01 14:39:29.483416
next_book_datetime=2023-11-01 14:40:59.483399
now=2023-11-01 14:39:29.483416
next_book_datetime=2023-11-01 14:40:59.483399
master counter = 33, stride = 81.25, class_idx = 3
'33','Yoga Tone 2' by 'Una',FULL,'2023-11-01 14:40:05.764300'
next_book_datetime=2023-11-01 14:41:10.764279
click_btn=None

master counter = 33, stride = 81.25, class_idx = 3
'33','Yoga Tone 2' by 'Una',FULL,'2023-11-01 14:40:05.764300'
next_book_datetime=2023-11-01 14:41:10.764279
click_btn=None

now=2023-11-01 14:40:11.771092
next_book_datetime=2023-11

master counter = 48, stride = 81.25, class_idx = 3
'48','Flow Yoga 動瑜珈' by 'Moorthi',FULL,'2023-11-01 14:45:11.699947'
next_book_datetime=2023-11-01 14:46:16.699930
click_btn=None

master counter = 48, stride = 81.25, class_idx = 3
'48','Flow Yoga 動瑜珈' by 'Moorthi',FULL,'2023-11-01 14:45:11.699947'
next_book_datetime=2023-11-01 14:46:16.699930
click_btn=None

master counter = 48, stride = 81.25, class_idx = 3
'48','Yoga Wheel 瑜珈輪' by 'Una',FULL,'2023-11-01 14:45:16.736169'
next_book_datetime=2023-11-01 14:46:21.736149
click_btn=None

master counter = 48, stride = 81.25, class_idx = 3
'48','Yoga Wheel 瑜珈輪' by 'Una',FULL,'2023-11-01 14:45:16.736169'
next_book_datetime=2023-11-01 14:46:21.736149
click_btn=None

now=2023-11-01 14:45:18.090576
next_book_datetime=2023-11-01 14:46:48.090561
now=2023-11-01 14:45:18.090576
next_book_datetime=2023-11-01 14:46:48.090561
now=2023-11-01 14:45:22.905125
next_book_datetime=2023-11-01 14:46:32.905111
now=2023-11-01 14:45:22.905125
next_book_datetime=2

master counter = 64, stride = 81.25, class_idx = 4
'64','Yoga Tone 2' by 'Una',FULL,'2023-11-01 14:50:43.556042'
next_book_datetime=2023-11-01 14:51:48.556026
click_btn=None

now=2023-11-01 14:50:48.699889
next_book_datetime=2023-11-01 14:52:08.699871
now=2023-11-01 14:50:48.699889
next_book_datetime=2023-11-01 14:52:08.699871
master counter = 65, stride = 81.25, class_idx = 5
'65','Flow Yoga 動瑜珈' by 'Moorthi',FULL,'2023-11-01 14:51:00.176348'
next_book_datetime=2023-11-01 14:52:05.176329
click_btn=None

master counter = 65, stride = 81.25, class_idx = 5
'65','Flow Yoga 動瑜珈' by 'Moorthi',FULL,'2023-11-01 14:51:00.176348'
next_book_datetime=2023-11-01 14:52:05.176329
click_btn=None

master counter = 65, stride = 81.25, class_idx = 5
'65','Yin Yoga 陰瑜珈' by 'Moorthi',FULL,'2023-11-01 14:51:00.517003'
next_book_datetime=2023-11-01 14:52:05.516982
click_btn=None

master counter = 65, stride = 81.25, class_idx = 5
'65','Yin Yoga 陰瑜珈' by 'Moorthi',FULL,'2023-11-01 14:51:00.517003'
next_book_d

now=2023-11-01 14:56:14.598443
next_book_datetime=2023-11-01 14:57:14.598428
now=2023-11-01 14:56:14.598443
next_book_datetime=2023-11-01 14:57:14.598428
master counter = 81, stride = 81.25, class_idx = 1
'81','Yoga Wheel 瑜珈輪' by 'Una',FULL,'2023-11-01 14:56:28.086487'
next_book_datetime=2023-11-01 14:57:33.086470
click_btn=None

master counter = 81, stride = 81.25, class_idx = 1
'81','Yoga Wheel 瑜珈輪' by 'Una',FULL,'2023-11-01 14:56:28.086487'
next_book_datetime=2023-11-01 14:57:33.086470
click_btn=None

now=2023-11-01 14:56:33.516701
next_book_datetime=2023-11-01 14:57:43.516684
now=2023-11-01 14:56:33.516701
next_book_datetime=2023-11-01 14:57:43.516684
master counter = 82, stride = 81.25, class_idx = 2
'82','Flow Yoga 動瑜珈' by 'Moorthi',FULL,'2023-11-01 14:56:51.150169'
next_book_datetime=2023-11-01 14:57:56.150151
click_btn=None

master counter = 82, stride = 81.25, class_idx = 2
'82','Flow Yoga 動瑜珈' by 'Moorthi',FULL,'2023-11-01 14:56:51.150169'
next_book_datetime=2023-11-01 14:57:

now=2023-11-01 15:01:41.502883
next_book_datetime=2023-11-01 15:03:21.502863
master counter = 97, stride = 81.25, class_idx = 2
'97','Hatha Basic 哈達基礎' by 'Una',FULL,'2023-11-01 15:01:52.764294'
next_book_datetime=2023-11-01 15:02:57.764277
click_btn=None

master counter = 97, stride = 81.25, class_idx = 2
'97','Hatha Basic 哈達基礎' by 'Una',FULL,'2023-11-01 15:01:52.764294'
next_book_datetime=2023-11-01 15:02:57.764277
click_btn=None

now=2023-11-01 15:01:57.526003
next_book_datetime=2023-11-01 15:02:57.525987
now=2023-11-01 15:01:57.526003
next_book_datetime=2023-11-01 15:02:57.525987
master counter = 98, stride = 81.25, class_idx = 3
'98','Flow Yoga 動瑜珈' by 'Moorthi',FULL,'2023-11-01 15:02:38.283210'
next_book_datetime=2023-11-01 15:03:43.283194
click_btn=None

master counter = 98, stride = 81.25, class_idx = 3
'98','Flow Yoga 動瑜珈' by 'Moorthi',FULL,'2023-11-01 15:02:38.283210'
next_book_datetime=2023-11-01 15:03:43.283194
click_btn=None

now=2023-11-01 15:02:43.445294
next_book_dateti